<a href="https://colab.research.google.com/github/Kirans1ngh/Machine-Learning-practice/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np

In [8]:
# A class to represent our neural network
class SimpleNeuralNetwork:
    """
    A simple Neural Network with one hidden layer.

    Attributes:
        input_nodes (int): Number of neurons in the input layer.
        hidden_nodes (int): Number of neurons in the hidden layer.
        output_nodes (int): Number of neurons in the output layer.
        learning_rate (float): The step size for updating weights and biases.
        weights_ih (np.array): Weights matrix from Input to Hidden layer.
        weights_ho (np.array): Weights matrix from Hidden to Output layer.
        bias_h (np.array): Bias vector for the Hidden layer.
        bias_o (np.array): Bias vector for the Output layer.
    """

    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate=0.1):
        """Initializes the network's architecture and parameters."""

        # --- 1. Define the architecture ---
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate

        # --- 2. Initialize weights and biases with random values ---
        # Weights between Input and Hidden layers
        # Shape: (number_of_hidden_nodes, number_of_input_nodes)
        self.weights_ih = np.random.rand(self.hidden_nodes, self.input_nodes) - 0.5

        # Weights between Hidden and Output layers
        # Shape: (number_of_output_nodes, number_of_hidden_nodes)
        self.weights_ho = np.random.rand(self.output_nodes, self.hidden_nodes) - 0.5

        # Biases for the hidden layer
        # Shape: (number_of_hidden_nodes, 1)
        self.bias_h = np.random.rand(self.hidden_nodes, 1) - 0.5

        # Biases for the output layer
        # Shape: (number_of_output_nodes, 1)
        self.bias_o = np.random.rand(self.output_nodes, 1) - 0.5

    def _sigmoid(self, x):
        """The Sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        """The derivative of the Sigmoid function."""
        # Note: x is assumed to be the output of a sigmoid function already
        return x * (1 - x)

    def predict(self, inputs_list):
        """Performs forward propagation to make a prediction."""

        # Convert input list to a 2D numpy array (a column vector)
        inputs = np.array(inputs_list, ndmin=2).T

        # --- FORWARD PROPAGATION ---

        # 1. Calculate signals into the hidden layer
        hidden_inputs = np.dot(self.weights_ih, inputs) + self.bias_h
        # 2. Calculate the signals emerging from the hidden layer
        hidden_outputs = self._sigmoid(hidden_inputs)

        # 3. Calculate signals into the final output layer
        final_inputs = np.dot(self.weights_ho, hidden_outputs) + self.bias_o
        # 4. Calculate the signals emerging from the final output layer
        final_outputs = self._sigmoid(final_inputs)

        return final_outputs

    def train(self, inputs_list, targets_list):
        """Trains the network by performing one full cycle of fwd/bwd propagation."""

        # Convert input and target lists to 2D numpy arrays
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        # --- 1. FORWARD PROPAGATION (same as predict, but we need intermediate values) ---
        hidden_inputs = np.dot(self.weights_ih, inputs) + self.bias_h
        hidden_outputs = self._sigmoid(hidden_inputs)

        final_inputs = np.dot(self.weights_ho, hidden_outputs) + self.bias_o
        final_outputs = self._sigmoid(final_inputs)

        # --- 2. BACKWARD PROPAGATION (The Learning Part) ---

        # Calculate the error (target - actual)
        output_errors = targets - final_outputs

        # Calculate the gradient for the output layer
        # Gradient = (Error * Derivative of Activation Function)
        output_gradient = self._sigmoid_derivative(final_outputs) * output_errors

        # Calculate the error for the hidden layer (propagated backward)
        hidden_errors = np.dot(self.weights_ho.T, output_errors)

        # Calculate the gradient for the hidden layer
        hidden_gradient = self._sigmoid_derivative(hidden_outputs) * hidden_errors

        # --- 3. UPDATE WEIGHTS AND BIASES ---

        # Update weights for the links between the hidden and output layers
        self.weights_ho += self.learning_rate * np.dot(output_gradient, hidden_outputs.T)

        # Update weights for the links between the input and hidden layers
        self.weights_ih += self.learning_rate * np.dot(hidden_gradient, inputs.T)

        # Update the biases
        self.bias_o += self.learning_rate * output_gradient
        self.bias_h += self.learning_rate * hidden_gradient

In [9]:
# Create an instance of the 3-2-1 network
nn_321 = SimpleNeuralNetwork(input_nodes=3, hidden_nodes=2, output_nodes=1, learning_rate=0.2)

# Create some dummy training data.
# The inputs are "normalized" to be between 0 and 1.
# Each row is a training example: [area, bedrooms, age]
training_data_inputs = np.array([
    [0.8, 0.1, 0.9],  # Input for target 0.1 (High area, low beds, old -> low price)
    [0.2, 0.8, 0.7],  # Input for target 0.7 (Low area, high beds, old -> mid price)
    [0.9, 0.8, 0.2]   # Input for target 0.9 (High area, high beds, new -> high price)
])
# Each row is the corresponding target output: [price_score]
training_data_targets = np.array([
    [0.1],
    [0.7],
    [0.9]
])

print("Initial Input-to-Hidden weights:\n", nn_321.weights_ih)
print("\nInitial Hidden-to-Output weights:\n", nn_321.weights_ho)

Initial Input-to-Hidden weights:
 [[ 0.21569994 -0.07292586 -0.27127706]
 [-0.24363736 -0.37229801 -0.40420386]]

Initial Hidden-to-Output weights:
 [[ 0.47665621 -0.25752168]]


In [10]:
# Set the number of training cycles
epochs = 5000

print("Training the 3-2-1 network...")
for e in range(epochs):
    # Loop through each record in our training data
    for i in range(len(training_data_inputs)):
        nn_321.train(training_data_inputs[i], training_data_targets[i])

print("Training complete!")

Training the 3-2-1 network...
Training complete!


In [11]:
# Test the network with a new, unseen situation
test_input = [0.85, 0.6, 0.3] # High area, mid beds, fairly new house
prediction = nn_321.predict(test_input)

print(f"Test Input: {test_input}")
print(f"Prediction (should be high, close to 0.9): {prediction[0][0]:.4f}")

Test Input: [0.85, 0.6, 0.3]
Prediction (should be high, close to 0.9): 0.8241


In [12]:
# --- Create, Train, and Test a 4-3-2 Network ---

# 1. Create the instance
nn_432 = SimpleNeuralNetwork(input_nodes=4, hidden_nodes=3, output_nodes=2, learning_rate=0.2)

# 2. Create the dummy data
training_data_inputs_432 = np.array([
    # area, beds, age, dist_city
    [0.9, 0.8, 0.2, 0.1], # Should result in: High price, fast sell
    [0.2, 0.3, 0.9, 0.8]  # Should result in: Low price, slow sell
])
# Targets: [price_score, sell_time_score] (high score = fast sell)
training_data_targets_432 = np.array([
    [0.9, 0.9],
    [0.2, 0.1]
])

# 3. Train the network
print("\nTraining the 4-3-2 network...")
epochs = 5000
for e in range(epochs):
     for i in range(len(training_data_inputs_432)):
        nn_432.train(training_data_inputs_432[i], training_data_targets_432[i])
print("Training complete!")


# 4. Test the network
test_input_432 = [0.7, 0.7, 0.4, 0.3] # Good house, pretty close to city
prediction_432 = nn_432.predict(test_input_432)

print(f"\nTest Input: {test_input_432}")
print(f"Prediction (Price, Sell Time) (should be high for both):")
print(f"  - Predicted Price Score: {prediction_432[0][0]:.4f}")
print(f"  - Predicted Sell Time Score: {prediction_432[1][0]:.4f}")


Training the 4-3-2 network...
Training complete!

Test Input: [0.7, 0.7, 0.4, 0.3]
Prediction (Price, Sell Time) (should be high for both):
  - Predicted Price Score: 0.8231
  - Predicted Sell Time Score: 0.7983
